# Portfolio 
## Andrea Mejia 
#### (858) 466-9040 
#### Email me at betterbiz.drea@gmail.com
This notebook to demonstrate some of my skillsets in python. This Project will focus on demonstrating data mining, web scraping, data analytics and basic machine learning. 

I will be demonstrating an analysis of twitter post. 

In [ ]:
#Imports
